In [44]:
# -*- coding: utf8 -*-
#########################IMPORTS############################################
from os.path import expanduser
import codecs, optparse
import re
import random as rd
import sys,os,time
import string
import yaml, YamlDuplicates
import ParFuMor as PFM
from ParFuMor import *
import pickle

In [45]:
from string import translate, maketrans

In [46]:
home = expanduser("~")
annee="21"
def getSerie(nomKalaba):
#     return home+"/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/Kalaba-Project/%s-%s/"%(annee,nomKalaba)
    return home+"/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/00-ProjetKalaba/%s-%s/"%(annee,nomKalaba)


prevSerie=getSerie("K2")
nextSerie=getSerie("K1")

nomsPropres=["Agathos","Clemencia","Elphaba","Freja","Hoderi","Kaleb","Mahira","Violette","Nabil","Katisha","Nicole"]
nomsPropresP=["agaTos","klemensja","elfaba","freZa","oderi","kaleb","maira","vjolet","nabil","katiSa","nikol"]

In [47]:
with open(prevSerie+"Stems.yaml", 'r') as stream:
    prevStems=yaml.safe_load(stream)
with open(nextSerie+"Stems.yaml", 'r') as stream:
    nextStems=yaml.safe_load(stream)

In [48]:
def getLeaves(stemStructure):
    formes={}
    for element in stemStructure:
        if type(stemStructure[element])==list:
            formesCle=", ".join(stemStructure[element])
            formes[formesCle]=element
        else:
            formes.update(getLeaves(stemStructure[element]))
    return formes

In [49]:
prevLexique=getLeaves(prevStems)
nextLexique=getLeaves(nextStems)

In [50]:
def replace_keys(old_dict, key_dict):
    new_dict = { }
    for key in old_dict.keys():
        new_key = key_dict.get(key, key)
        if isinstance(old_dict[key], dict):
            new_dict[new_key] = replace_keys(old_dict[key], key_dict)
        else:
            new_dict[new_key] = old_dict[key]
    return new_dict


In [51]:
with open(prevSerie+"Phonology.yaml", 'r') as stream:
    phonology=yaml.safe_load(stream)

In [52]:
obstrusivesSourdes=u"ptkfTsS"
obstrusivesVoisees=u"bdgvDzZ"

voisement=maketrans(obstrusivesSourdes,obstrusivesVoisees)
devoisement=maketrans(obstrusivesVoisees,obstrusivesSourdes)


consonnes=phonology["consonnes"]
voyelles=phonology["voyelles"]

In [53]:
def devoiserFinale(radical):
    if radical[-1] in obstrusivesVoisees:
        return radical.translate(devoisement)
    else:
        return None

def voiserVCV(radical):
    result=list(radical)
    cibles=[]
    for p,l in enumerate(radical[1:-1]):
        if l in obstrusivesSourdes and radical[p] in voyelles and radical[p+2] in voyelles:
            cibles.append(p+1)
    if cibles:
        cible=rd.choice(cibles)
        result[cible]=result[cible].translate(voisement)
        return "".join(result)
    else:
        return None

def metathese(radical):
    result=list(radical)
    cibles=[]
    for p,l in enumerate(radical):
        if l in consonnes:
            cibles.append(p)
    if len(cibles)>1 and rd.randint(1,3)==1:
        cible1=rd.choice(cibles[:-1])
        cible2=cibles[cibles.index(cible1)+1]
        result[cible1]=radical[cible2]
        result[cible2]=radical[cible1]
        result="".join(result)
        if result!=radical:
            return result
        else:
            return None
    else:
        return None

    
def apophonie(radical,choice="rdVowel"):
    result=list(radical)
    cibles=[]
    if not choice:
        choice=rd.choice(["apophonie","rdVowel"])
    if choice=="apophonie":
        for p,l in enumerate(radical):
            if l in "iae":
                cibles.append(p)
        if cibles:
            cible=rd.choice(cibles)
            result[cible]=phonology["apophonies"][result[cible]]
            return "".join(result)
        else:
            return None
    else:
        for p,l in enumerate(radical):
            if l in voyelles:
                cibles.append(p)
        if cibles:
            cible=rd.choice(cibles)
            result[cible]=rd.choice(voyelles)
            return "".join(result)
        else:
            return None

def changerLongueur(radical):
    result=list(radical)
    if len(radical)==5:
        if radical[0] in consonnes:
            if radical[-1] in consonnes:
                if rd.randint(1,5)==1:
                    result.append(rd.choice(voyelles))
                    return "".join(result)
                else:
                    return None
            else:
                if rd.randint(1,5)==1:
                    result.append(rd.choice(consonnes))
                    return "".join(result)
                else:
                    return None
        else:
            if rd.randint(1,3)==1:
                result.insert(0,rd.choice(consonnes))
                return "".join(result)
            else:
                return None
    elif len(radical)>5:
        if rd.randint(1,3)==1:
            if radical[0] in consonnes:
                result=result[:-1]
            else:
                result=result[1:]
            return "".join(result)
        else:
            return None
    else:
        if rd.randint(1,3)==1:
            if radical[0] in voyelles:
                result.insert(0,rd.choice(consonnes))
            elif radical[-1] in voyelles:
                result.append(rd.choice(consonnes))
            else:
                result.append(rd.choice(voyelles))
            return "".join(result)
        else:
            return None
        
            

In [54]:
apophonie("kulubu",choice="rnd")

'kulobu'

In [55]:
operations=[
    devoiserFinale,
    apophonie,apophonie,
    voiserVCV,voiserVCV,
    metathese,
    changerLongueur
           ]

In [56]:
def modifierRadical(radical):
    nOperations=rd.choice([0,1,1,1,2,2,2,3,3,3,4,5])
    while nOperations>0:
        operation=rd.choice(operations)
        result=operation(radical)
        if result:
            radical=result
            nOperations-=1
    return radical

In [57]:
racineChanges={}

print "lexique suivant"
for mot in prevLexique:
    if mot in nextLexique:
        if prevLexique[mot] in nomsPropresP:
            racineChanges[nextLexique[mot]]=prevLexique[mot]
        else:
            racineChanges[nextLexique[mot]]=modifierRadical(prevLexique[mot])
#         print prevLexique[mot],racineChanges[nextLexique[mot]]
    else:
#         print "***1"
#         print mot,"n'est pas dans le nouveau lexique"
#         print "***"
        print mot.split(",")[0]
print "================="
print "lexique précédent"
for mot in nextLexique:
    if not mot in prevLexique:
#         print "***"
#         print mot,u"n'était pas dans le lexique précédent"
#         print "***"
        print mot.split(",")[0]



lexique suivant
chasseur
villageois
loup
enfant
lexique précédent
villageois
garçon
chasseur
enfant
loup


In [58]:
for i in range(10):
    print modifierRadical("Tonif")

Tonef
Tonuf
Tonifo
noTif
Tunifi
Tonif
noDof
Tunof
Tanif
Tonefe


In [59]:
racineChanges

{'Dadejar': 'nuvaS',
 'Dafib': 'TimeZo',
 'DamuS': 'puniku',
 'Dazep': 'DagiN',
 'DiNe': 'foDug',
 'Dojaf': 'lopade',
 'DomeZ': 'tituf',
 'DotoSut': 'beZiT',
 'DuTog': 'pagaT',
 'DuZob': 'duSuv',
 'Dubir': 'kotob',
 'Duvop': 'vezep',
 'NaTat': 'baged',
 'NaZad': 'pulaj',
 'NabiS': 'ZeZij',
 'Nawazut': 'piDol',
 'NeZoro': 'nidis',
 'Negegiz': 'bagak',
 'Neranuf': 'maduDa',
 'Nikad': 'vibadi',
 'Nivumil': 'tudek',
 'NoTim': 'fivaT',
 'NunaDas': 'Sidubi',
 'NutoS': 'gomaS',
 'Sagan': 'riZad',
 'Sagobul': 'luDuru',
 'Segeg': 'meZulu',
 'SejuT': 'potesa',
 'Senaf': 'voDas',
 'Sif': 'luvoga',
 'SoDim': 'wajedo',
 'Sofaw': 'Nabet',
 'Sogeges': 'widiN',
 'Sorit': 'Namut',
 'Sozup': 'kifim',
 'SutaN': 'Zavari',
 'TaDak': 'ledip',
 'TezuT': 'gaDol',
 'TiledaS': 'piNop',
 'TiniSum': 'vuTer',
 'Tipej': 'Sagit',
 'Tizok': 'tonure',
 'ToT': 'kip',
 'Togoniw': 'boman',
 'Tuseg': 'Tibaf',
 'Zavag': 'DuDul',
 'Zebuf': 'riliN',
 'ZipiS': 'Nubof',
 'Zuges': 'bowap',
 'Zuru': 'fobuda',
 'Zutig': 'Tarit',


In [61]:
newStems=replace_keys(nextStems,racineChanges)

In [62]:
with open(nextSerie+'flowStems.yaml', 'w') as output:
    yaml.dump(newStems, output, default_flow_style=False,allow_unicode=True)

In [63]:
with open(nextSerie+"flowStems.yaml", 'r') as input:
    yamlLines=input.readlines()

In [64]:
yamlText="".join(yamlLines)
yamlText=re.sub(r"\n\s*-\s*",", ",yamlText)
yamlText=re.sub(r":,\s*",": ",yamlText)
yamlText=re.sub(r": *([^\n]+)",": [\g<1>]",yamlText)
print yamlText

ADJ:
  Nalok: [deux]
  Spok: [six]
  TaraZo: [bas, basse, basses]
  bageN: [héroïque, héroïques]
  begut: [nombreux, nombreuses]
  bikin: [jaune, jaunes]
  bowap: [disparu, disparus, disparue, disparues]
  degeT: [nouveau, nouveaux, nouvelle, nouvelles]
  forok: [courageux, courageuse, courageuses]
  gabur: [trois]
  genar: [effrayé, effrayés, effrayée, effrayées]
  kifim: [différent, différents, différente, différentes]
  koron: [troisième, troisièmes]
  lidip: [gros, grosse, grosses]
  menaN: [quatre]
  mumino: [petit, petits, petite, petites]
  nadeDe: [vert, verts, verte, vertes]
  nukuk: [cinq]
  pagaT: [méchant, méchants, méchante, méchantes]
  pakas: [profond, profonds, profonde, profondes]
  pigok: [maigre, maigres]
  pomak: [furieux, furieuse, furieuses]
  povuv: [blanc, blancs, blanche, blanches]
  pulaj: [sept]
  puniku: [autre, autres]
  robenu: [vilain, vilains, vilaine, vilaines]
  ronuk: [noir, noirs, noire, noires]
  rowor: [dernier, derniers, dernière, dernières]
  rus

In [65]:
with open(nextSerie+'newStems.yaml', 'w') as output:
    output.write(yamlText)